In [4]:
import pandas as pd
from mtgoScraper import *


augOutput = mtgoScrape.formatDeckList('standard',2025,8)
skipUrls = ["/decklist/standard-challenge-32-2025-08-0112806287", "/decklist/standard-league-2025-08-019495"]
urlList = [x['url'] for x in augOutput if x['url'] not in skipUrls]

Searching for standard deck lists, 2025-08.


In [20]:
dfA = mtgoScrape.getDeckListsFromUrlList(urlList)

Trying to load /decklist/standard-challenge-32-2025-08-1612810061 from previously downloaded  decklist.
                                          index  Quantity  \
Deck    Main/Side Card Name                                 
Deck 0  Main      Agatha's Soul Cauldron      0         4   
                  Fear of Missing Out         1         4   
                  Marauding Mako              2         4   
                  Proft's Eidetic Memory      3         4   
                  Spirebluff Canal            4         4   
...                                         ...       ...   
Deck 31 Side      Magebane Lizard            19         2   
                  Obliterating Bolt          20         2   
                  Ruinous Rampage            21         2   
                  Soul-Guide Lantern         22         2   
                  Suplex                     23         1   

                                                                                   Deck URL  
Deck    

In [301]:
dfB = dfA
dfB = dfB.reset_index()
dfB = dfB[dfB['Main/Side'] == 'Main']
dfB = dfB.set_index(['Deck URL', 'Deck'])
dfB = dfB.sort_index()
deckLists = dfB.index.unique()

jaccThreshold = 0.5

uniqueDeckCounter = 0
uniqueDeckDict = {0: deckLists[0]} # Base Case

def deckComparisonPrep(deck1, deck2):
    deck1 = deck1.reset_index()
    deck2 = deck2.reset_index()
    merged = pd.merge(deck1, deck2, how='outer')
    merged = merged.pivot(index='Card Name', columns=['Deck URL', 'Deck'], values='Quantity')
    return merged

def getJaccard(deck1id,deck2id):
    deck1 = dfB.loc[deck1id].sort_index()
    deck2 = dfB.loc[deck2id].sort_index()
    df = deckComparisonPrep(deck1, deck2)
    df = df.fillna(0)
    df['Union'] = df[[deck1id,deck2id]].max(axis=1)
    df['Intersect'] = df[[deck1id,deck2id]].min(axis=1)
    jaccardVal = df['Intersect'].sum() / df['Union'].sum()
    print(deck1id, deck2id, jaccardVal)
    return jaccardVal

for i in range(1, len(deckLists)):
    uniqueDeckKeys = uniqueDeckDict.keys()
    tempDict = uniqueDeckDict.copy()
    jaccVals = []
    for key in uniqueDeckKeys:
        jaccVal = getJaccard(deckLists[i],uniqueDeckDict[key])
        jaccVals += [jaccVal]
        if jaccVal > jaccThreshold:
            break
    if [x for x in jaccVals if x > jaccThreshold] == []:
        uniqueDeckCounter += 1
        tempDict[uniqueDeckCounter] = deckLists[i]
        print(tempDict)
    uniqueDeckDict = tempDict


('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 1') ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 0') 0.17647058823529413
{0: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 0'), 1: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 1')}
('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 10') ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 0') 0.08108108108108109
('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 10') ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 1') 0.034482758620689655
{0: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 0'), 1: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 1'), 2: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 10')}
('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 11') ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 0') 0.875
('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 1

In [305]:
uniqueDeckDict

{0: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 0'),
 1: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 1'),
 2: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 10'),
 3: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 12'),
 4: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 15'),
 5: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 17'),
 6: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 18'),
 7: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 2'),
 8: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 22'),
 9: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 25'),
 10: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 28'),
 11: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 6'),
 12: ('/decklist/standard-challenge-32-2025-08-0112806297', 'Deck 7'),
 13: ('/decklist/standard-challenge-32-2025-08-0212806304', 'Deck 11'),
 14: ('

In [328]:
dfC = dfB.loc[deckLists[1]]
dfC['Card Name'] = dfC['Card Name'].str.strip().str.lower()
merged = pd.merge(dfC, oracleDf[scryKeepCols], how='left', left_on='Card Name', right_on='name')

def getDeckColour(deckDict):
    return deckDict['color_identity'].unique()

merged['color_identity']

C:\Users\knath\AppData\Local\Temp\ipykernel_23528\947791308.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfC['Card Name'] = dfC['Card Name'].str.strip().str.lower()


0        [B]
1        [B]
2        [U]
3        [U]
4     [B, U]
5        [U]
6        [U]
7     [B, U]
8     [B, U]
9         []
10       [B]
11       [B]
12    [B, W]
13    [B, U]
14        []
15       [U]
16       [B]
17       [B]
18       [U]
19       [B]
Name: color_identity, dtype: object

In [44]:
oracleDfO = bulkOracle()

In [181]:
oracleDf = oracleDfO
oracleDf = oracleDf[~oracleDf['layout'].str.contains("art_series")]
oracleDf = oracleDf[~oracleDf['layout'].str.contains("token")]
oracleDf['name'] = oracleDf['name'].str.strip().str.lower()
oracleDf['name'] = oracleDf['name'].str.split(' // ').str[0]
oracleDf

,object,id,oracle_id,multiverse_ids,mtgo_id,tcgplayer_id,cardmarket_id,name,lang,released_at,...,produced_mana,tcgplayer_etched_id,loyalty,life_modifier,hand_modifier,attraction_lights,color_indicator,flavor_name,content_warning,defense
0,card,a471b306-4941-4e46-a0cb-d92895c16f8a,00037840-6089-42ec-8c5c-281f9f474504,[692174],137223.0,615195.0,807933.0,"nissa, worldsoul speaker",en,2025-02-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,86bf43b1-8d4e-4759-bb2d-0b2e03ba7012,0004ebd0-dfd6-4276-b4a6-de0003e94237,[15862],15870.0,3094.0,3081.0,static orb,en,2001-04-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,7050735c-b232-47a6-a342-01795bfd0d46,0006faf6-7a61-426c-9034-579f2cfcfa83,[370780],49283.0,69965.0,262945.0,sensory deprivation,en,2013-07-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,e718b21b-46d1-4844-985c-52745657b1ac,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,[470580],77122.0,196536.0,391692.0,road of return,en,2019-08-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,036ef8c9-72ac-46ce-af07-83b79d736538,000d5588-5a4c-434e-988d-396632ade42c,[83282],22609.0,12835.0,12551.0,storm crow,en,2005-07-29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35585,card,d6695fa8-881c-407c-91d9-3ac770372d35,fffa9334-3576-4f70-9605-2ad062cdbc69,[430770],64646.0,136683.0,298834.0,without weakness,en,2017-07-14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35586,card,4d5edf4f-5695-42fc-9e57-c4faef60fbc3,fffaa634-42a0-4038-b027-24f28754fec7,[571548],101914.0,276869.0,665173.0,firesong and sunspeaker,en,2022-07-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,card,22a6a5f1-1405-4efb-af3e-e1f58d664e99,fffcb71f-e802-436e-917d-eaa6607ab74f,"[503766, 503767]",87643.0,230214.0,530502.0,"toralf, god of fury",en,2021-02-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35588,card,09f06f55-7918-46c4-80ff-0bf39e091a4a,fffdc2ac-bde4-4e4c-a5bd-0e6c6e49ad91,[430833],64772.0,132234.0,298282.0,"samut, the tested",en,2017-07-14,...,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
oracleDf[oracleDf['name'].str.contains('aclazotz,')]

def expand_faces(row):
    if isinstance(row.get('card_faces'), list):
        front = row['card_faces'][0]
        back = row['card_faces'][1] if len(row['card_faces']) > 1 else {}
        row['mana_cost'] = front.get('mana_cost')
        row['type_line'] = front.get('type_line')
        row['oracle_text'] = front.get('oracle_text')
        row['back_name'] = back.get('name')
        x = back.get('type_line')
        row['back_type_line'] = x
        if "Creature" in x:
            print(front.get('name'),back.get('name'))
            row['back_power'] = back.get('power')
            row['back_toughness'] = back.get('toughness')
        row['back_type_line'] = back.get('type_line')
        row['back_oracle_text'] = back.get('oracle_text')
    return row

oracleDf = oracleDf.apply(expand_faces, axis=1)


Ulvenwald Captive Ulvenwald Abomination
Azusa's Many Journeys Likeness of the Seeker
The Emperor of Palamecia The Lord Master of Hell
Westvale Abbey Ormendahl, Profane Prince
Lunarch Veteran Luminous Phantom
Seraph of New Capenna Seraph of New Phyrexia
Extricator of Sin Extricator of Flesh
Tales of Master Seshiro Seshiro's Living Legacy
Devoted Grafkeeper Departed Soulkeeper
Alrund, God of the Cosmos Hakka, Whispering Raven
Life of Toshiro Umezawa Memory of Toshiro
Sun-Blessed Guardian Furnace-Blessed Conqueror
Graveyard Trespasser Graveyard Glutton
Henrika Domnathi Henrika, Infernal Seer
Ulrich of the Krallenhorde Ulrich, Uncontested Alpha
Shady Traveler Stalking Predator
Lambholt Elder Silverpelt Werewolf
Nezumi Graverobber Nighteyes the Desecrator
Overwhelmed Archivist Archive Haunt
Tovolar's Huntmaster Tovolar's Packleader
Ravenous Demon Archdemon of Greed
Mystic Skull Mystic Monstrosity
Invasion of Azgol Ashen Reaper
Kaslem's Stonetree Kaslem's Strider
Hostile Hostel Creeping Inn


In [186]:
oracleDf[oracleDf['name'].str.contains('angel')]

,all_parts,arena_id,artist,artist_ids,attraction_lights,back_name,back_oracle_text,back_power,back_toughness,back_type_line,...,set_uri,story_spotlight,tcgplayer_etched_id,tcgplayer_id,textless,toughness,type_line,uri,variation,watermark
65,NaN,83920.0,Igor Kieryluk,[9c3e9d17-509f-485c-9360-46d897ce716b],NaN,NaN,NaN,NaN,NaN,NaN,...,https://api.scryfall.com/sets/04bef644-343f-42...,False,NaN,478930.0,False,4,Creature — Phyrexian Cleric,https://api.scryfall.com/cards/626c46a3-72b8-4...,False,NaN
326,NaN,82055.0,Miguel Mercado,[085f314d-cd61-48e0-89d1-048e9891e4a1],NaN,NaN,NaN,NaN,NaN,NaN,...,https://api.scryfall.com/sets/4e47a6cd-cdeb-4b...,False,NaN,282765.0,False,4,Creature — Angel,https://api.scryfall.com/cards/2d00bab2-e95d-4...,False,NaN
345,NaN,NaN,Todd Lockwood,[d0b9f3cc-13fb-42b6-bc43-b034728e5c7b],NaN,NaN,NaN,NaN,NaN,NaN,...,https://api.scryfall.com/sets/2ec77b94-6d47-48...,False,NaN,180835.0,False,5,Creature — Angel,https://api.scryfall.com/cards/a07809b2-dabe-4...,False,NaN
436,NaN,73184.0,Ryan Pancoast,[89cc9475-dda2-4d13-bf88-54b92867a25c],NaN,NaN,NaN,NaN,NaN,NaN,...,https://api.scryfall.com/sets/f4e01fa7-b254-42...,False,NaN,221948.0,False,6,Creature — Angel Cleric,https://api.scryfall.com/cards/9897074a-0ac4-4...,False,NaN
823,NaN,NaN,Adam Rex,[445bd2fb-4a70-4599-87a4-83cf5979fc56],NaN,NaN,NaN,NaN,NaN,NaN,...,https://api.scryfall.com/sets/a29e8ace-bbcd-45...,False,NaN,58062.0,False,NaN,Enchantment,https://api.scryfall.com/cards/b5c268cc-891b-4...,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35101,NaN,NaN,Jeff Miracola,[9b465bde-3656-4495-ace4-af4ce29999ed],NaN,NaN,NaN,NaN,NaN,NaN,...,https://api.scryfall.com/sets/30ec97cb-dca9-4b...,False,NaN,15446.0,False,4,Creature — Shapeshifter,https://api.scryfall.com/cards/24f0c586-a1a5-4...,False,NaN
35162,NaN,69129.0,Ryan Yee,[8955dca7-3e37-42b4-83a9-167c78a2178f],NaN,NaN,NaN,NaN,NaN,NaN,...,https://api.scryfall.com/sets/97a7fd84-8d89-45...,False,NaN,183201.0,False,4,Creature — Angel,https://api.scryfall.com/cards/80164e61-3e94-4...,False,NaN
35363,NaN,NaN,Sal Villagran,[0a0b2cc1-67dd-4dec-9e41-49af469012cb],NaN,NaN,NaN,NaN,NaN,NaN,...,https://api.scryfall.com/sets/02a67a0d-b4da-4c...,False,NaN,93563.0,False,NaN,Instant,https://api.scryfall.com/cards/dfdf70c7-49e2-4...,False,NaN
35500,NaN,90348.0,Denys Tsiperko,[5d145909-04a3-442d-a664-c7fdb166e2b0],NaN,NaN,NaN,NaN,NaN,NaN,...,https://api.scryfall.com/sets/55a85ebe-644e-4b...,False,NaN,544425.0,False,5,Creature — Angel,https://api.scryfall.com/cards/c853d04c-864b-4...,False,NaN


In [312]:
df = dfA.reset_index()
#df = df.set_index(['Main/Side', 'Card Name'])
df = df.groupby(['Main/Side', 'Card Name'])['Quantity'].sum()
df = df.sort_values(ascending=False)
df = df.reset_index()
df = df.pivot(index='Card Name', columns='Main/Side', values='Quantity').fillna(0).reset_index()
df['Card Name'] = df['Card Name'].str.strip().str.lower()
scryKeepCols = ['name','mana_cost','type_line','oracle_text','power','toughness','colors','color_identity','set','rarity','back_name', 'back_type_line', 'back_oracle_text', 'back_power', 'back_toughness']
df = pd.merge(df,oracleDf[scryKeepCols], left_on='Card Name', right_on='name', how='left')

df

,Card Name,Main,Side,name,mana_cost,type_line,oracle_text,power,toughness,colors,color_identity,set,rarity,back_name,back_type_line,back_oracle_text,back_power,back_toughness
0,abhorrent oculus,12.0,2.0,abhorrent oculus,{2}{U},Creature — Eye,"As an additional cost to cast this spell, exil...",5,5,[U],[U],dsk,mythic,NaN,NaN,NaN,NaN,NaN
1,abrade,609.0,535.0,abrade,{1}{R},Instant,Choose one —\n• Abrade deals 3 damage to targe...,NaN,NaN,[R],[R],tdc,uncommon,NaN,NaN,NaN,NaN,NaN
2,absolute virtue,1.0,0.0,absolute virtue,{6}{W}{U},Legendary Creature — Avatar Warrior,This spell can't be countered.\nFlying\nYou ha...,8,8,"[U, W]","[U, W]",fin,mythic,NaN,NaN,NaN,NaN,NaN
3,"aclazotz, deepest betrayal",3.0,15.0,"aclazotz, deepest betrayal",{3}{B}{B},Legendary Creature — Bat God,"Flying, lifelink\nWhenever Aclazotz attacks, e...",NaN,NaN,NaN,[B],lci,mythic,Temple of the Dead,Land,"(Transforms from Aclazotz, Deepest Betrayal.)\...",NaN,NaN
4,"adagia, windswept bastion",18.0,0.0,"adagia, windswept bastion",,Land — Planet,This land enters tapped.\n{T}: Add {W}.\nStati...,NaN,NaN,[],[W],peoe,mythic,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620,"zahur, glory's past",2.0,0.0,"zahur, glory's past",{W}{B},Legendary Creature — Zombie Cat Warrior,"Start your engines! (If you have no speed, it ...",3,2,"[B, W]","[B, W]",dft,rare,NaN,NaN,NaN,NaN,NaN
621,zero point ballad,0.0,48.0,zero point ballad,{X}{B},Sorcery,Destroy all creatures with toughness X or less...,NaN,NaN,[B],[B],eoe,rare,NaN,NaN,NaN,NaN,NaN
622,zoetic glyph,4.0,0.0,zoetic glyph,{2}{U},Enchantment — Aura,Enchant artifact\nEnchanted artifact is a Gole...,NaN,NaN,[U],[U],lci,uncommon,NaN,NaN,NaN,NaN,NaN
623,zombify,16.0,0.0,zombify,{3}{B},Sorcery,Return target creature card from your graveyar...,NaN,NaN,[B],[B],fdn,uncommon,NaN,NaN,NaN,NaN,NaN


In [88]:
missing = df[df['type_line'].isnull()]
print("Unmatched cards:", missing['Card Name'].unique())

Unmatched cards: ['aclazotz, deepest betrayal' 'bloomvine regent' 'braided net'
 'cease/desist' 'cecil, dark knight' 'cheeky house-mouse'
 'clay-fired bricks' 'disruptive stormbrood' 'elusive otter'
 'emet-selch, unsundered' 'esper origins' 'horned loch-whale'
 "imodane's recruiter" 'ishgard, the holy see' "jill, shiva's dominant"
 "joshua, phoenix's dominant" 'marang river regent' 'mosswood dreadknight'
 'ojer axonil, deepest might' 'questing druid' "saheeli's lattice"
 'scavenger regent' 'sephiroth, fabled soldier' 'shrouded shepherd'
 'sidequest: raise a chocobo' 'spring-loaded sawblades'
 'terra, magical adept' 'the enigma jewel' 'thousand moons smithy'
 'twinmaw stormbrood']
